## Packages and Functions

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import math
import scipy
import decimal

In [2]:
def neg_bin(x, r, p):
    numerator = math.factorial(x + (r -1))
    denom = math.factorial(x) * math.factorial(r - 1)
    multiplier = math.pow(p, x) * math.pow(1 - p, r)
    return (numerator / denom) * multiplier

def margin(x):
    percent = (x/3) + (5/3)
    return (1+(percent / 100))


## Match List Inputs

In [3]:
path_to_data = '../data/match_list.csv'
match_list = pd.read_csv(path_to_data , usecols = [1,2,3,4,5,6] )
match_list

,Match_ID,Home,Away,Sell,Buy,R
0,0,Tottenham,Liverpoo,10.0,10.5,120
1,1,Bournemouth,Leicester,10.0,10.0,120
2,2,Chelsea,Cardiff,10.0,10.5,120
3,3,Huddersfield,Palace,10.0,10.5,120
4,4,Man City,Fulham,10.0,10.5,120
5,5,Newcastle,Arsenal,10.0,10.5,120
6,6,Watford,Man United,10.0,10.5,120
7,7,Wolves,Burnley,10.0,10.5,120
8,8,Everton,West Ham,10.0,10.5,120
9,9,Southampton,Brighton,10.0,10.0,120


In [4]:
n=1 # DONT FORGET TO CHANGE!!!!
home = 'Bournemouth'
away = 'Leicester'
input_sell = 10
input_buy = 10
input_R = 120
match_list.loc[n] = [n , home , away , input_sell, input_buy , input_R]
match_list.to_csv('../data/match_list.csv')

## Calculate Fair Values

In [5]:
model_prices = pd.read_csv('../data/model_prices' , usecols = [1,2,3,4,5,6,7,8,9,10,11,12])
exact = pd.DataFrame(data = ({'X' : list(range(0,31,1)) , 'prob_sell' : list(range(0,31,1)), 'prob_buy' : list(range(0,31,1))}))
over = pd.DataFrame(data = ({'X' : list(range(0,31,1)) , 'prob' : list(range(0,31,1)), 'price' : list(range(0,31,1))}))
under = pd.DataFrame(data = ({'X' : list(range(0,31,1)) , 'prob' : list(range(0,31,1)), 'price' : list(range(0,31,1))}))

In [6]:
for i, row in match_list.iterrows():

    R = match_list.R[i] 
    sell = match_list.Sell[i] 
    buy = match_list.Buy[i] 

    p_sell = (sell/R) *(1/(1+(sell/R)))
    p_buy = (buy/R)*(1/(1+(buy/R)))
    
    exact['prob_sell'] = exact.apply(lambda i: neg_bin(i['X'], R, p_sell), axis=1)
    exact['prob_buy'] = exact.apply(lambda i: neg_bin(i['X'], R , p_buy), axis=1)
    over['prob'] = 1 - exact.prob_sell.cumsum()
    over['price'] = 1/over.prob
    under['prob'] = exact.prob_buy.cumsum() - exact.prob_buy
    under['price'] = 1/under.prob

    model_prices.loc[i] = [match_list.Match_ID[i] , under.price[10] , 1/(exact.prob_sell[10] + exact.prob_sell[11] + exact.prob_sell[12]) , over.price[12] , under.price[14] , over.price[13] , under.price[9] , over.price[8] , under.price[11] , over.price[10] , under.price[6] , over.price[5]]
    model_prices.to_csv('../data/model_prices')    
 

## Apply Margin


In [7]:
order_prices = pd.DataFrame(model_prices)

In [8]:
order_prices['9_or_under'] = order_prices['9_or_under'] * margin(order_prices['9_or_under'])
order_prices['10_to_12'] = order_prices['10_to_12'] * margin(order_prices['10_to_12'])
order_prices['13_or_more'] = order_prices['13_or_more'] * margin(order_prices['13_or_more'])
order_prices['Under_13.5'] = order_prices['Under_13.5'] * margin(order_prices['Under_13.5'])
order_prices['Over_13.5'] = order_prices['Over_13.5'] * margin(order_prices['Over_13.5'])
order_prices['Under_10.5'] = order_prices['Under_10.5'] * margin(order_prices['Under_10.5'])
order_prices['Over_10.5'] = order_prices['Over_10.5'] * margin(order_prices['Over_10.5'])
##order_prices['Under_8.5'] = order_prices['Under_8.5'] * margin(order_prices['Under_8.5'])
order_prices['Over_8.5'] = order_prices['Over_8.5'] * margin(order_prices['Over_8.5'])
order_prices['Under_5.5'] = order_prices['Under_5.5'] * margin(order_prices['Under_5.5'])
order_prices['Over_5.5'] = order_prices['Over_5.5'] * margin(order_prices['Over_5.5'])



In [9]:
model_prices.head()

,Match_ID,9_or_under,10_to_12,13_or_more,Under_13.5,Over_13.5,Under_8.5,Over_8.5,Under_10.5,Over_10.5,Under_5.5,Over_5.5
0,0.0,2.533865,3.197306,4.781216,1.249756,7.210766,3.448706,1.552273,1.954792,2.457852,18.711119,1.102897
1,1.0,2.211801,3.197306,4.781216,1.192522,7.210766,2.925825,1.552273,1.753339,2.457852,14.164233,1.102897
2,2.0,2.533865,3.197306,4.781216,1.249756,7.210766,3.448706,1.552273,1.954792,2.457852,18.711119,1.102897
3,3.0,2.533865,3.197306,4.781216,1.249756,7.210766,3.448706,1.552273,1.954792,2.457852,18.711119,1.102897
4,4.0,2.533865,3.197306,4.781216,1.249756,7.210766,3.448706,1.552273,1.954792,2.457852,18.711119,1.102897


In [10]:
order_prices.head()

,Match_ID,9_or_under,10_to_12,13_or_more,Under_13.5,Over_13.5,Under_8.5,Over_8.5,Under_10.5,Over_10.5,Under_5.5,Over_5.5
0,0.0,2.533865,3.197306,4.781216,1.249756,7.210766,3.448706,1.552273,1.954792,2.457852,18.711119,1.102897
1,1.0,2.211801,3.197306,4.781216,1.192522,7.210766,2.925825,1.552273,1.753339,2.457852,14.164233,1.102897
2,2.0,2.533865,3.197306,4.781216,1.249756,7.210766,3.448706,1.552273,1.954792,2.457852,18.711119,1.102897
3,3.0,2.533865,3.197306,4.781216,1.249756,7.210766,3.448706,1.552273,1.954792,2.457852,18.711119,1.102897
4,4.0,2.533865,3.197306,4.781216,1.249756,7.210766,3.448706,1.552273,1.954792,2.457852,18.711119,1.102897


## Staking

In [12]:
stake = pd.DataFrame(order_prices)
        